In [1]:
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')
%config InlineBackend.figure_format = 'retina'
%pylab inline
from __future__ import division
import rdflib
from rdflib.graph import Graph, ConjunctiveGraph
from rdflib import Graph, URIRef, BNode, Literal
from rdflib import RDF
from rdflib import Namespace
from rdflib.namespace import OWL, RDF, RDFS, FOAF, XSD
from SPARQLWrapper import SPARQLWrapper, JSON
import numpy as np
import matplotlib.pyplot as plt

Populating the interactive namespace from numpy and matplotlib


# Table of Contents
* [1. Define Graph](#1.-Define-Graph)
* [2. Retrieve Data](#2.-Retrieve-Data)
    * [2.1 Country](#2.1-Country)
    * [2.2 Cantons](#2.2-Cantons)
    * [2.2 Plot Correlation](#2.2-Plot-Correlation)

# 1 Define Graph

In [2]:
g = Graph()

schema = rdflib.Namespace('http://schema.org/')
time   = rdflib.Namespace('http://www.w3.org/2006/time#')
dbo    = rdflib.Namespace('http://dbpedia.org/ontology/')
dbp    = rdflib.Namespace('http://dbpedia.org/property/')
res    = rdflib.Namespace('http://dbpedia.org/resource/')
rdf    = rdflib.Namespace('http://www.w3.org/1999/02/22-rdf-syntax-ns#')
rdfs   = rdflib.Namespace('http://www.w3.org/2000/01/rdf-schema#')
g.parse(dbo)
# Add the OWL data to the graph
# g.add((cName, RDF.type, dbo.country))

<Graph identifier=Nb51ccaf4c860407ca0331502612b92e0 (<class 'rdflib.graph.Graph'>)>

In [3]:
g.add(( res['Switzerland'], RDF.type, dbo.Country))

# 2 Retrieve Data

## 2.1 Country

In [4]:
sparql = SPARQLWrapper("http://dbpedia.org/sparql")
query = """
    PREFIX dbo:  <http://dbpedia.org/ontology/>
    PREFIX dbpedia: <http://dbpedia.org/resource/>
    PREFIX dbp: <http://dbpedia.org/property/>
    PREFIX geo: <http://www.w3.org/2003/01/geo/wgs84_pos#>
    SELECT DISTINCT  ?s ?pop ?code
    WHERE{ 
       ?s rdf:type dbo:PopulatedPlace.
       ?s geo:lat ?lat.
       ?s geo:lat ?lon.
       ?s dbo:populationTotal ?pop.
       ?s geo:geometry ?geom.
       ?s dbp:iso3166code ?code.
       FILTER (?s = dbpedia:Switzerland)
    }
"""
sparql.setQuery(query)
sparql.setReturnFormat(JSON)
results = sparql.query().convert()

In [9]:
sparql = SPARQLWrapper("http://dbpedia.org/sparql")
query = """
    PREFIX dbo: <http://dbpedia.org/ontology/>
    PREFIX dbr: <http://dbpedia.org/resource/>
    SELECT ?s WHERE {
     ?s a dbo:City ;
        dbo:country dbr:Switzerland
    }
"""
sparql.setQuery(query)
sparql.setReturnFormat(JSON)
results = sparql.query().convert()

In [10]:
for result in results["results"]["bindings"]:
    print result["s"]["value"]

http://dbpedia.org/resource/Lausanne
http://dbpedia.org/resource/Uster
http://dbpedia.org/resource/Biel/Bienne
http://dbpedia.org/resource/Burgdorf,_Switzerland
http://dbpedia.org/resource/Lucerne
http://dbpedia.org/resource/Lugano
http://dbpedia.org/resource/Schaffhausen
http://dbpedia.org/resource/Thun
http://dbpedia.org/resource/Winterthur
http://dbpedia.org/resource/Dietikon
http://dbpedia.org/resource/La_Chaux-de-Fonds
http://dbpedia.org/resource/Zofingen
http://dbpedia.org/resource/Emmen,_Switzerland
http://dbpedia.org/resource/Steinhausen,_Switzerland
http://dbpedia.org/resource/Zürich


In [5]:
for result in results["results"]["bindings"]:
    N = int(result["pop"]["value"])

In [6]:
N

8341000

In [7]:
for result in results["results"]["bindings"]:
    N = int(result["pop"]["value"])
totPopulation   = Literal(N)
g.add(( res['Switzerland'], dbo.populationTotal, totPopulation ))
g.add(( res['Switzerland'], dbp.iso3166code, Literal(str(result["code"]["value"])) ))

In [ ]:
g.serialize(destination='outputProva.nt', format='nt')

## 2.2 Cantons

In [8]:
### Retrieve data from Official Statistical office
### Retrieve Information Of Cantons
sparql = SPARQLWrapper("https://sparql.geo.admin.ch/sparql")
query = """
PREFIX schema: <http://schema.org/>
PREFIX gn: <http://www.geonames.org/ontology#>
PREFIX geo: <http://www.opengis.net/ont/geosparql#>
select ?Canton ?Population ?Coords ?cantonName ?bfsNumber
where
{
    ?Canton a gn:A.ADM1 .
    ?Canton schema:name ?cantonName .
    ?Canton gn:population ?Population .
    ?Canton geo:hasGeometry ?Geometry .
    ?Geometry geo:asWKT ?Coords.
    ?Canton <https://ld.geo.admin.ch/def/bfsNumber> ?bfsNumber.
    ?Canton <http://purl.org/dc/terms/issued> ?Date .
    FILTER (?Date = "2016-01-01"^^xsd:date)
}
ORDER BY DESC(?Population)
"""
sparql.setQuery(query)
sparql.setReturnFormat(JSON)
results = sparql.query().convert()

In [ ]:

g.add((tmp, RDF.type, dbo.Canton ))

In [10]:
c = 0
for result in results["results"]["bindings"]:
    tmp = result["Canton"]['value']
    uri = 
    g.add((tmp, RDF.type, dbo.Canton ))

AssertionError: Subject https://ld.geo.admin.ch/boundaries/canton/1:2016 must be an rdflib term

In [ ]:
c = 0
for result in results["results"]["bindings"]:
    c = c + 1
    #tmp = Literal(c)
    #g.add((tmp, RDF.type, dbo.Canton))
    #pop = int(result["Population"]["value"])
    #pop = Literal(pop)
    #g.add((tmp, dbo.populationTotal, pop ))

In [ ]:
g.serialize(destination='outputProva.nt', format='nt')

In [ ]:
result["cantonName"]["value"]

In [ ]:
for result in results["results"]["bindings"]:
    g.add((result["Canton"], RDF.type, dbo.Canton ))
    print result["cantonName"]

In [ ]:
result["Canton"]

In [ ]:
g.parse("http://dbpedia.org/")

qres = g.query(
    """    
    PREFIX dbo:  <http://dbpedia.org/ontology/>
    PREFIX dbpedia: <http://dbpedia.org/resource/>
    PREFIX dbp: <http://dbpedia.org/property/>
    PREFIX geo: <http://www.w3.org/2003/01/geo/wgs84_pos#>
    SELECT DISTINCT  ?s ?pop ?code
    WHERE{ 
       ?s rdf:type dbo:PopulatedPlace.
       ?s geo:lat ?lat.
       ?s geo:lat ?lon.
       ?s dbo:populationTotal ?pop.
       ?s geo:geometry ?geom.
       ?s dbp:iso3166code ?code.
       FILTER (?s = dbpedia:Switzerland)
    }""")

In [ ]:
for row in qres:
    print("%s %s %s" %row)


In [ ]:
qres

In [ ]:
sparql = SPARQLWrapper("http://dbpedia.org/sparql")
query = """
    PREFIX dbo:  <http://dbpedia.org/ontology/>
    PREFIX dbpedia: <http://dbpedia.org/resource/>
    PREFIX dbp: <http://dbpedia.org/property/>
    PREFIX geo: <http://www.w3.org/2003/01/geo/wgs84_pos#>
    SELECT DISTINCT  ?s ?pop ?code
    WHERE{ 
       ?s rdf:type dbo:PopulatedPlace.
       ?s geo:lat ?lat.
       ?s geo:lat ?lon.
       ?s dbo:populationTotal ?pop.
       ?s geo:geometry ?geom.
       ?s dbp:iso3166code ?code.
       FILTER (?s = dbpedia:Switzerland)
    }
"""
sparql.setQuery(query)
sparql.setReturnFormat(JSON)
#results = sparql.query().convert()

In [ ]:
for result in results["results"]["bindings"]:
    print result["s"]["value"], result["pop"]["value"]

In [ ]:
N = int(results["pop"]["value"])
totPopulation   = Literal(N)
g.add(( cName, dbo.populationTotal, totPopulation ))
g.add(( cName, dbp.iso3166code, Literal(str(result["code"]["value"])) ))

In [ ]:
g.serialize(destination='outputProva.nt', format='nt')

## 2.2 Cantons

In [ ]:
### Retrieve data from Official Statistical office
### Retrieve Information Of Cantons
sparql = SPARQLWrapper("https://sparql.geo.admin.ch/sparql")
query = """
PREFIX schema: <http://schema.org/>
PREFIX gn: <http://www.geonames.org/ontology#>
PREFIX geo: <http://www.opengis.net/ont/geosparql#>
select ?Canton ?Population ?Coords ?cantonName ?bfsNumber
where
{
    ?Canton a gn:A.ADM1 .
    ?Canton schema:name ?cantonName .
    ?Canton gn:population ?Population .
    ?Canton geo:hasGeometry ?Geometry .
    ?Geometry geo:asWKT ?Coords.
    ?Canton <https://ld.geo.admin.ch/def/bfsNumber> ?bfsNumber.
    ?Canton <http://purl.org/dc/terms/issued> ?Date .
    FILTER (?Date = "2016-01-01"^^xsd:date)
}
ORDER BY DESC(?Population)
"""
sparql.setQuery(query)
sparql.setReturnFormat(JSON)
results = sparql.query().convert()

In [ ]:
fig,ax = plt.subplots(figsize=(10,10))
ax.bar(result["cantonName"]["value"], result["Population"]["value"], align='center', alpha=0.5)


In [ ]:
for result in results["results"]["bindings"]:
    print result["Population"]["value"], result["cantonName"]["value"], result["bfsNumber"]["value"]

In [ ]:
### Retrieve data from Official Statistical office
### Retrieve Information Of Municipality
sparql = SPARQLWrapper("https://sparql.geo.admin.ch/sparql")
query = """
PREFIX schema: <http://schema.org/>
PREFIX gn: <http://www.geonames.org/ontology#>
PREFIX geo: <http://www.opengis.net/ont/geosparql#>
select ?Municipality ?Name ?Population ?Coords ?Canton ?cantonName
where
{
    ?Municipality a gn:A.ADM3 .
    ?Municipality schema:name ?Name .
    ?Municipality gn:population ?Population .
    ?Municipality geo:hasGeometry ?Geometry .
    ?Geometry geo:asWKT ?Coords. 
    ?Municipality gn:parentADM1 ?Canton .
    ?Canton schema:name ?cantonName .
    ?Municipality <http://purl.org/dc/terms/issued> ?Date .
    ?Municipality <https://ld.geo.admin.ch/def/bfsNumber> ?bfsNumber . #connect to the ?bfsNumber found in wikidata.
    FILTER (?Date = "2017-01-01"^^xsd:date)
}
LIMIT 5
"""
sparql.setQuery(query)
sparql.setReturnFormat(JSON)
results = sparql.query().convert()

In [ ]:
for result in results["results"]["bindings"]:
    print result["Name"]["value"]

In [ ]:
### Retrieve data from Official Statistical office
### Retrieve Information Of Municipality
sparql = SPARQLWrapper("https://sparql.geo.admin.ch/sparql")
query = """
PREFIX schema: <http://schema.org/>
PREFIX gn: <http://www.geonames.org/ontology#>
PREFIX geo: <http://www.opengis.net/ont/geosparql#>
select ?Municipality ?Name ?Population ?Coords ?Canton ?cantonName
where
{
    ?Municipality a gn:A.ADM3 .
    ?Municipality schema:name ?Name .
    ?Municipality gn:population ?Population .
    ?Municipality geo:hasGeometry ?Geometry .
    ?Geometry geo:asWKT ?Coords. 
    ?Municipality gn:parentADM1 ?Canton .
    ?Canton schema:name ?cantonName .
    ?Municipality <http://purl.org/dc/terms/issued> ?Date .
    ?Municipality <https://ld.geo.admin.ch/def/bfsNumber> ?bfsNumber . #connect to the ?bfsNumber found in wikidata.
    FILTER (?Date = "2017-01-01"^^xsd:date)
}
LIMIT 5
"""
sparql.setQuery(query)
sparql.setReturnFormat(JSON)

In [ ]:
results = sparql.query().convert()

In [ ]:
for result in results["results"]["bindings"]:
    print result["Name"]["value"], result["Population"]["value"], result["cantonName"]["value"]

In [ ]:
### Retrieve data from Official Statistical office
### Retrieve Information Of Municipality
sparql = SPARQLWrapper("https://sparql.geo.admin.ch/sparql")
query = """
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX wdt: <http://www.wikidata.org/prop/direct/>
PREFIX wikibase: <http://wikiba.se/ontology#>
PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
PREFIX wd: <http://www.wikidata.org/entity/>
PREFIX fn: <http://www.w3.org/2005/xpath-functions#>
PREFIX schema: <http://schema.org/>
PREFIX gn: <http://www.geonames.org/ontology#>
PREFIX geo: <http://www.opengis.net/ont/geosparql#>

SELECT ?Municipality ?Name  ?zipCode 
WHERE {
    ?Municipality a schema:AdministrativeArea . #specify only the non-versioned entries.
    ?Municipality schema:name ?Name .
    ?Municipality a gn:A.ADM3 . #municipality only
    #?Municipality gn:population ?Population .    
    #?Municipality <http://purl.org/dc/terms/issued> ?Date .
    #FILTER (?Date = "2017-01-01"^^xsd:date)
    ?Municipality geo:hasGeometry ?Geometry .
    ?Municipality <https://ld.geo.admin.ch/def/bfsNumber> ?bfsNumber . #connect to the ?bfsNumber found in wikidata.
    {
    SELECT DISTINCT (xsd:integer(?bfs) AS ?bfsNumber) ?WikidataURI ?GeoNamesURI ?zipCode WHERE {
    SERVICE <https://query.wikidata.org/bigdata/namespace/wdq/sparql>
    {
        ?WikidataURI wdt:P771 ?bfs . #look for bfs number in WD
        ?WikidataURI wdt:P31 wd:Q70208 . #municipality of Switzerland
        ?WikidataURI wdt:P1566 ?GeoNamesId . 
        ?WikidataURI wdt:P281 ?zipCode . 
    }
      BIND(IRI(CONCAT(STR("http://sws.geonames.org/"), ?GeoNamesId, "/")) AS ?GeoNamesURI)
    }
  }
}
ORDER BY ?Name
"""
sparql.setQuery(query)
sparql.setReturnFormat(JSON)

In [ ]:
results = sparql.query().convert()

In [ ]:
for result in results["results"]["bindings"]:
    print result["Name"]["value"], result["zipCode"]["value"]